In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
src = os.path.join(parentdir,'src')
sys.path.insert(0,src) 
import functions
import pandas as pd
import numpy as np
import geopandas as gpd
import psycopg2

import cv2
from skimage import img_as_ubyte
import os
import sys
import time
import scipy.ndimage as nd
import matplotlib.pyplot as plt

%matplotlib tk
plt.ion() # set interactive mpl
plt.rcParams["image.cmap"] = "gist_gray"

In [2]:
def getPhoto(photoURL):
    photoName = photoURL.split('/')[-1]
    os.system('wget ' + photoURL)
    img = nd.imread(photoName)
    os.system('rm ' + photoName)
    return img

def photoCroping(photoURL):
    img = getPhoto(photoURL)
    
    #number of rows and number of col
    nrow, ncol = img.shape[:2]
    #clipping from half the picture to 90% of the picture removes
    #the street ahead and also the bike tire
    croppedPicture = img[nrow//2:(nrow-nrow//10),:,:]
    return croppedPicture

def photoPlot(img):
    nrow, ncol = img.shape[:2]
    xsize = 10.
    ysize = xsize * float(nrow) / float(ncol)
    
    
    fig0, ax0 = plt.subplots(num=0, figsize=[xsize, ysize])
    fig0.subplots_adjust(0, 0, 1, 1)
    ax0.axis("off")
    im0 = ax0.imshow(img)
    fig0.canvas.draw()

In [3]:
def photoPlot_w_caption(img, img2, txt, num):
    plt.subplot(121),plt.imshow(img, cmap = 'gray')
    plt.title('Original Mara'), plt.xticks([]), plt.yticks([])
    plt.subplot(122),plt.imshow(img2, cmap = 'gray')
    plt.title('Defected Mara'), plt.xticks([]), plt.yticks([])
    plt.show()

In [32]:
def filter_out_line_markings(img):
    '''
    goal is to filter on color to eliminate line markings
    a lot of code taken from here:
    http://docs.opencv.org/trunk/df/d9d/tutorial_py_colorspaces.html
    '''
    lower = np.array([0, 10, 50])
    upper = np.array([360, 100, 100])
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hls, lower, upper)
    res = cv2.bitwise_and(hls, hls, mask = mask)
    return res

In [28]:
#importing base image
img = nd.imread('/Users/geoffperrin/Desktop/CUSP/capstone/computer_vision/mara/12890022.JPG')
    
    #determine pothole / crack score
md = nd.filters.median_filter

    #blur photo
md_blurPhoto = md(img, 5)

In [30]:
img.shape

(2376, 3583, 3)

In [33]:
#filter out white line markings
photo_no_white = filter_out_line_markings(md_blurPhoto)

In [34]:
photoPlot(photo_no_white)

In [35]:
edges_cv = cv2.Canny(md_blurPhoto, 200, 400)

#blur edges
blurred_edges = cv2.GaussianBlur(edges_cv,(3,3),0)

    #only want to keep cracks that are near other cracks or that have a minimum threshold
bdilation = nd.morphology.binary_dilation
berosion = nd.morphology.binary_erosion
edges_2 = bdilation(berosion(blurred_edges, iterations=2), iterations=2)

In [36]:
photoPlot(edges_2)

In [37]:
photoPlot_w_caption(img, edges_2, "", 0)

In [38]:
edges_2.mean()

0.0050062209216549152